# Imputation of Missing Values for Wind Stations

When taking measurements with devices, their malfunctioning and failure need to be considered. 
The collected measurements often end up being incomplete and with missing values (actual NaN). 
In order to extract the full information from these devices, YData's Missing Imputer is coming to rescue. 

For this use case we are going to work with wind measurements.
The data consists in a time series composed by multiple wind wands. The wands are 10 in total, and all of them have a certain amount of data that is missing. 
Also, 2 devices have no records at all. (Cold Start Meters). <br>
With the help of YData's Algorithms we are going to impute the missing values within the time series and upsample their granularity in order to have more information on what happens between the hours. We will also be able to generate months into the future that have not been observed yet and recreate from zero cold start meters.
<br> <br>
<div align="center">
    <img align='center' width="650" height="300" src="diagram.png">
</div>


## 0 - Setup


### 0.1 - Imports

Load the necessary dependencies.

In [2]:
%%capture 
!pip install ipywidgets 

In [3]:
%%capture
from typing import List
from pandas import concat
from ipywidgets import interact

# YDATA Proprietary Lib
from ydata.dataset import Dataset
from ydata.connectors import GCSConnector
from ydata.utils.formats import read_json
from ydata.quality.impute.timeseries import TSMissingImputer

### 0.2 - Auxiliary Functions

The auxiliary functions are custom-designed utilities developed for this use case.

In [4]:
from factors import save_json
from utils import setting_index_data
from results import meter_statistics
from upsample import upsample_stations
from preprocess import preprocess_data
from imputation import get_cold_start_meters, get_proxy_data, resample_station_data, data_boundaries, load_factors, resample
from results_viz import plot_timeseries, compare_acf, plot_multicolor_line, plot_hist, compare_lags, plot_metric_byperiod, polar_graph

## 1 - Load Data
Time Span of Data: 26 November 2018 - 1 March 2021

In [5]:
# Load the credentials
credentials = read_json('gcs_credentials.json')

# Create the connector for Google Cloud Storage
connector = GCSConnector('ydatasynthetic', gcs_credentials=credentials)

# Read the first extraction data 26 Nov 2020 to 15 Jul 2021
data = connector.read_file('gs://pipelines_artifacts/wind_measurements_pipeline/data/ethiopia_wind_lot_nan.csv')

## 2 - Data Processing

_Train_: 26 November 2018 - 1 March 2021

_Holdout_: from 2 March 2021 onwards 

Steps:
  1. Select the relevant columns
  2. Convert to Pandas DataFrame
  3. Cast to the correct data types
  4. Join the readings from both extractions into a single table
  5. Split into train and holdout

In [6]:
train, holdout = preprocess_data(data)

### 2.1 - Data Before Imputation

Visualize the data over time, before the imputation of missing values. 

In [7]:
@interact(meter=set(train['station']))
def plot_meter_data(meter):
    meter_original = train[train['station']==meter][:8000]
    print(' \n Percentage of missing data: {percentage:.2f}%'.format(percentage=meter_original.speed.isna().sum()/len(meter_original)*100))
    plot_timeseries(data=meter_original, feature='speed', title='Original Data for '+ meter)

interactive(children=(Dropdown(description='meter', options=('tuluguled1', 'seladingay', 'mega', 'sure', 'kebr…

As you can see, the data is all fractioned and full of gaps.

## 3 - Calculate Factors 

In order to have more information about the behaviour of the data we calculate the average windspeed per month for meters in relevant provinces. These values will help the algorithm to better model the data.

In [8]:
YEARS = [2018, 2019, 2020, 2021]
# Internal IDs that identify stations which are in the same provinces
meter_ids = read_json('meter_ids.json')

In [9]:
def get_monthly_avg_per_year(connector: GCSConnector, meter_ids: List, years=YEARS):
    "Calculates the monthly factor over the anual average, per year, for meters within same provinces as the original data."

    # create a map of each month to corresponding index
    meses = ['january', 'february', 'march', 'april', 'may', 'june', 'july',
             'august', 'september', 'october', 'november', 'december']

    months = {k : v for (k, v) in zip(meses, range(1, len(meses) + 1))}

    factors = []  # will contain a Series of monthly averages over anual, per each year

    for year in years:
        filepath = f'gs://pipelines_artifacts/wind_measurements_pipeline/data/df_for_factors_{str(year)}.csv'
        df = connector.read_file(filepath, assume_missing=True).to_pandas().set_index('name_station')  # read the yearly monthly averages
        df = df[df.index.isin(meter_ids)]                                                # filter for meters in same provinces
        df = df.dropna()                                                                 # drop the missing values
        for month in months.keys():                                                      # calculate factor as ratio of monthly average
            df[month] = df[month] / df['yearly']                                          # over the anual average
        factors.append(df[months.keys()].mean().copy())

    # Aggregate data
    agg_data = concat(factors, axis=1)
    agg_data.columns = years
    agg_data = agg_data.rename(index=months)
    agg_data['avg'] = agg_data.mean(axis=1)
    return agg_data

In [10]:
factors = get_monthly_avg_per_year(connector=connector, meter_ids=meter_ids)
factors = load_factors({'windspeed': factors['avg'].to_dict()})

## 4 - Imputation 

### 4.1 - Cold Start Meters

Training on cold-start meters (i.e. without any observed values) should be made in separate from the rest of the meters.

#### 4.1.1 - Data Wrangling 

In [11]:
# Get a list of cold-start meters
cold_start_meters = get_cold_start_meters(train)
cold_start_meters

['aysha1', 'gode1']

In [12]:
# Define the stations that can serve as proxy data for the cold-start meters
proxy_stations = {
    'aysha1': ['diredawa1', 'tuluguled1'],
    'gode1': ['seladingay', 'ziway'],
}
proxy_data = get_proxy_data(proxy_stations, train)

In [13]:
# Subset the data for cold-start meters only.
cold_start_data = train[train['station'].isin(cold_start_meters)]

#### 4.1.2 - Boundaries 

Apply the data boundaries to each dataframe used as proxy data for cold start in order to avoid out of bound values. 

In [14]:
proxy_data = {k: data_boundaries(v, replace_na=True) for (k,v) in proxy_data.items()}

#### 4.1.3 - Imputer 

The TSMissingImputer is responsible to impute the missing values for time-series.
- Learns the temporal dynamics from the observed values
- Supports multiple entities with the `partition_by` parameter
- Follows the usual scikit-learn method interfaces (e.g. fit, transform)

In [15]:
cold_imputer = TSMissingImputer()

# Train the Imputer
cold_imputer.fit(cold_start_data, partition_by='station', num_cols=['speed'], proxy_data=proxy_data, add_factors=factors)

TSMissingImputer()

#### 4.1.4 - Impute Full Year

Construct a full year of data, on hourly basis, for devices with observed readings. For each hour, the average of windspeed/winddirection is calculated and used as ground-truth for observed readings.

In [16]:
# Create a DataFrame of a whole year for all the meters with observed values.
whole_year = resample_station_data(cold_start_data)

# Apply the missing values imputation to reconstruct a whole year of data.
cold_reconstructed = cold_imputer.transform(whole_year)

#### 4.1.5 - Impute Holdout 

Construct a full month of holdout, on hourly basis, for devices with observed readings.

In [17]:
# Filter for cold-start meters on holdout data.
cold_holdout = holdout[holdout['station'].isin(cold_start_meters)]
whole_holdout = resample_station_data(cold_holdout, start_ts='2021-09-01', end_ts='2021-10-01')
cold_holdout_reconstructed = cold_imputer.transform(whole_holdout)

#### 4.1.6 - Data Validation

After the reconstruction check that no value is missing.

In [18]:
assert cold_reconstructed.isna().sum().sum() == 0, "The reconstructed dataset contains missing values after reconstruction."
assert cold_holdout_reconstructed.isna().sum().sum() == 0, "The reconstructed dataset of holdout contains missing values after reconstruction."

#### 4.1.7 - Post Processing 

The imputation of time-series is applicable to any type of numerical data and thus agnostic to energy-specific boundaries of wind measurements. To guarantee adequacy for wind speed and direction, we enforce that wind speed cannot be negative and that wind direction should range within degree angles (between 0 and 360).

In [19]:
# Postprocess the training data
postprocessed = data_boundaries(data=cold_reconstructed)

# Postprocess the holdout data
postprocessed_holdout = data_boundaries(data=cold_holdout_reconstructed)

### 4.2 - Standard Meters 

Meters with observed values. 

In [20]:
# Remove the cold-start meters from the train data.
train_data = train[~train['station'].isin(cold_start_meters)]

#### 4.2.1 - Data Boundaries 

In [21]:
train_data = data_boundaries(train_data, replace_na=True)

#### 4.2.2 - Imputer 

In [22]:
# Train the Imputer
imputer = TSMissingImputer()
# Train the Imputer
imputer.fit(train_data, partition_by='station', num_cols=['speed'], add_factors=factors)

TSMissingImputer()

#### 4.2.3 - Impute Full Year 

In [23]:
# Create a DataFrame of a whole year for all the meters with observed values.
whole_year = resample_station_data(train_data)

In [24]:
# Apply the missing values imputation to reconstruct a whole year of data.
reconstructed = imputer.transform(whole_year)

#### 4.2.4 - Impute Holdout  

In [25]:
# Remove the cold-start meters from holdout data.
standard_holdout = holdout[~holdout['station'].isin(cold_start_meters)]
whole_holdout = resample_station_data(standard_holdout, start_ts='2021-03-01', end_ts='2021-04-30')
holdout_reconstructed = imputer.transform(whole_holdout)

#### 4.2.5 - Data Validation

In [26]:
# After reconstruction, no value should be missing
assert reconstructed.isna().sum().sum() == 0, "The reconstructed dataset contains missing values after reconstruction."
assert holdout_reconstructed.isna().sum().sum() == 0, "The reconstructed dataset of holdout contains missing values after reconstruction."

#### 4.2.6 - Post Processing 

In [27]:
# Postprocess the training data
postprocessed = data_boundaries(data=reconstructed)

# Postprocess the holdout data
postprocessed_holdout = data_boundaries(data=holdout_reconstructed)

## 5 - Results Visualizations 

Comparison between original and generated data through visualizations. 

In [28]:
@interact(meter=set(reconstructed['station']))
def plot_meter_results(meter):
    meter_reconstructed = reconstructed[reconstructed['station']==meter][:8000]
    meter_original = train[train['station']==meter]
    #[:8000]

    # Convert the original data into resampled for full year 
    original_year = resample_station_data(meter_original)

    meter_statistics(original_year)
    plot_timeseries(data=original_year, feature='speed', title='Original Data for '+ meter)
    plot_multicolor_line(original_year, meter_reconstructed, title='Reconstruction of '+ meter)
    compare_acf(original_year, meter_reconstructed, title='Hourly Autocorrelation')
    plot_hist(original_year, meter_reconstructed, title='Distribution Overlap - Original and Reconstructed Series - '+ meter)
    compare_lags(original_year, meter_reconstructed, 'speed')
    plot_metric_byperiod(original_year, meter_reconstructed, 'speed')
    polar_graph(original_year, meter_reconstructed)

interactive(children=(Dropdown(description='meter', options=('tuluguled1', 'mega', 'seladingay', 'diredawa1', …

## 6 - Upsampling 

Upsample the measurements to 15-minutes frequency.  

In [29]:
# Load the reconstructed data for the given device
reconstructed_upsampled = upsample_stations(reconstructed)

In [30]:
# Resample the original data for 15 minute frequency
original_upsampled = resample(train, ts_col='timestamp', ts_freq='15T', partition_by='station')

## 7 - Store Data

Load data into the cloud. 

In [31]:
# Load the credentials
credentials = read_json('gcs_credentials.json')

# Create the connector for Google Cloud Storage
connector = GCSConnector('ydatasynthetic', gcs_credentials=credentials)

# Store the reconstructed data upsampled to 15 minutes
connector.write_file(reconstructed_upsampled, 'gs://pipelines_artifacts/wind_measurements_pipeline/outputs/reconstructed_upsampled.csv', index=True)

# Store the original data upsampled to 15 minutes
connector.write_file(original_upsampled, 'gs://pipelines_artifacts/wind_measurements_pipeline/outputs/original_upsampled.csv', index=True)